# Run this code to deploy an agent

In [2]:
import vertexai

PROJECT_ID = "virtual-nurse-450613"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://deploy-short-bot"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

from google.adk.agents import Agent
from vertexai.preview.reasoning_engines import AdkApp
import yfinance as yf

def get_stock_price(ticker: str):
    stock = yf.Ticker(ticker)
    price = stock.info.get("currentPrice", "Price not available")
    return {"price": price, "ticker": ticker}

root_agent = Agent(
    model='gemini-2.0-flash-001',
    name='root_agent',
    description='A simple agent that get stock price.',
    instruction="""You are a stock price assistant. Always use the get_stock_price tool.
                Include the ticker symbol in your response""",
    tools=[get_stock_price]
)

from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

session = app.create_session(user_id="u_123")

In [3]:
session

Session(id='36e687a6-bcdb-4f8d-b125-337f293df72d', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749667124.973777)

In [4]:
app.list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[Session(id='36e687a6-bcdb-4f8d-b125-337f293df72d', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749667124.973777)])

In [7]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="whats is Apple stock price?",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-9be9aaca-45d3-4d09-908a-8ea43cc8c83f', 'args': {'ticker': 'AAPL'}, 'name': 'get_stock_price'}}], 'role': 'model'}, 'invocation_id': 'e-f9411943-7961-4dae-8f08-4b461b969c74', 'author': 'root_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'L7dD6zr1', 'timestamp': 1749667218.300309}
{'content': {'parts': [{'function_response': {'id': 'adk-9be9aaca-45d3-4d09-908a-8ea43cc8c83f', 'name': 'get_stock_price', 'response': {'price': 199.4986, 'ticker': 'AAPL'}}}], 'role': 'user'}, 'invocation_id': 'e-f9411943-7961-4dae-8f08-4b461b969c74', 'author': 'root_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'tBP1iTMR', 'timestamp': 1749667222.636659}
{'content': {'parts': [{'text': 'The current stock price for Apple (AAPL) is $199.4986.\n'}], 'role': 'model'}, 'invocation_id': 'e-f9411943-7961-4dae-8f08-4b461b969c74', '

In [11]:
from vertexai import agent_engines

remote_app = agent_engines.create(
    agent_engine=root_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]", "yfinance"   
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'google-cloud-aiplatform': '1.94.0', 'pydantic': '2.11.5', 'cloudpickle': '3.1.1'}
The following requirements are missing: {'pydantic', 'cloudpickle'}
The following requirements are appended: {'cloudpickle==3.1.1', 'pydantic==2.11.5'}
The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'yfinance', 'cloudpickle==3.1.1', 'pydantic==2.11.5']
Using bucket deploy-short-bot
Wrote to gs://deploy-short-bot/agent_engine/agent_engine.pkl
Writing to gs://deploy-short-bot/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://deploy-short-bot/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/533873564462/locations/us-central1/reasoningEngines/8914989811589185536/operations/8648178849659486208
View progress and logs at https://console.cloud.google.com/logs/query?project=virtual-nurse-450613
AgentEngin

In [12]:
remote_app.resource_name

'projects/533873564462/locations/us-central1/reasoningEngines/8914989811589185536'

In [13]:
remote_session = remote_app.create_session(user_id="u_456")
remote_session

{'appName': '8914989811589185536',
 'events': [],
 'state': {},
 'id': '4268177145433423872',
 'lastUpdateTime': 1749668672.336696,
 'userId': 'u_456'}

In [14]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="what is Apple stock price?",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-08fec5c1-aad4-4557-8daf-7f21f08aa74c', 'args': {'ticker': 'AAPL'}, 'name': 'get_stock_price'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 8, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 8}], 'prompt_token_count': 71, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 71}], 'total_token_count': 79, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-81911d56-2358-4092-978e-b1642e80e964', 'author': 'root_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'hP4MPRXB', 'timestamp': 1749668720.760501}
{'content': {'parts': [{'function_response': {'id': 'adk-08fec5c1-aad4-4557-8daf-7f21f08aa74c', 'name': 'get_stock_price', 'response': {'price': 198.93, 'ticker': 'AAPL'}}}], 'role': 'user'}, 'invocation_id': 'e-81911d56-2358-4092-978e-b1642e80e964', 'author': 'root_agent', 'actions': {'state_delta': {}, 'artifact_de